In [10]:
import pandas as pd

# Load data
data = pd.read_csv('output.csv')

# Drop unnecessary columns
data = data.drop(columns=['X', 'Y', 'Z', 'Roll', 'Pitch', 'Yaw'])

# Select features and targets for BaseSpeed and DeltaSpeed models
features_base_speed = data[['Angle']]
target_base_speed = data['BaseSpeed']

features_delta_speed = data[['Error', 'DeltaError']]
target_delta_speed = data['DeltaSpeed']

In [11]:
from sklearn.model_selection import train_test_split

# Split data for BaseSpeed model
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(features_base_speed, target_base_speed, test_size=0.2, random_state=42)

# Split data for DeltaSpeed model
X_train_delta, X_test_delta, y_train_delta, y_test_delta = train_test_split(features_delta_speed, target_delta_speed, test_size=0.2, random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression

# Initialize models
model_base_speed = LinearRegression()
model_delta_speed = LinearRegression()

# Train models
model_base_speed.fit(X_train_base, y_train_base)
model_delta_speed.fit(X_train_delta, y_train_delta)

LinearRegression()

In [13]:
from sklearn.metrics import mean_squared_error, r2_score

# Predictions
y_pred_base = model_base_speed.predict(X_test_base)
y_pred_delta = model_delta_speed.predict(X_test_delta)

# Evaluation for BaseSpeed model
mse_base = mean_squared_error(y_test_base, y_pred_base)
r2_base = r2_score(y_test_base, y_pred_base)
print(f'BaseSpeed - Mean Squared Error: {mse_base}, R-squared: {r2_base}')

# Evaluation for DeltaSpeed model
mse_delta = mean_squared_error(y_test_delta, y_pred_delta)
r2_delta = r2_score(y_test_delta, y_pred_delta)
print(f'DeltaSpeed - Mean Squared Error: {mse_delta}, R-squared: {r2_delta}')

BaseSpeed - Mean Squared Error: 7.020446059922661e-07, R-squared: 0.9999994742874181
DeltaSpeed - Mean Squared Error: 0.2429488949877355, R-squared: 0.06925796527435157


In [14]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {'fit_intercept': [True, False], 'copy_X': [True, False], 'positive': [True, False]}

# Perform Grid Search for BaseSpeed model
grid_search_base = GridSearchCV(model_base_speed, param_grid, cv=5)
grid_search_base.fit(X_train_base, y_train_base)
print("Best parameters for BaseSpeed model:", grid_search_base.best_params_)

# Perform Grid Search for DeltaSpeed model
grid_search_delta = GridSearchCV(model_delta_speed, param_grid, cv=5)
grid_search_delta.fit(X_train_delta, y_train_delta)
print("Best parameters for DeltaSpeed model:", grid_search_delta.best_params_)

Best parameters for BaseSpeed model: {'copy_X': True, 'fit_intercept': True, 'positive': False}
Best parameters for DeltaSpeed model: {'copy_X': True, 'fit_intercept': True, 'positive': True}


In [15]:
# Retrain BaseSpeed model with best parameters
model_base_speed = LinearRegression(**grid_search_base.best_params_)
model_base_speed.fit(X_train_base, y_train_base)

# Retrain DeltaSpeed model with best parameters
model_delta_speed = LinearRegression(**grid_search_delta.best_params_)
model_delta_speed.fit(X_train_delta, y_train_delta)

LinearRegression(positive=True)

In [16]:
import joblib

# Save models
joblib.dump(model_base_speed, 'base_speed_model.pkl')
joblib.dump(model_delta_speed, 'delta_speed_model.pkl')

['delta_speed_model.pkl']